# Task 3: Nhận định "mùa vụ" của các combo sản phẩm

Notebook này thực hiện phân tích để xác định tính mùa vụ của các combo sản phẩm dựa trên luật kết hợp theo thời gian.

## Mục tiêu

- Phân tích luật kết hợp theo từng tháng/quý
- Xác định các combo sản phẩm có tính mùa vụ (biến thiên cao giữa các giai đoạn)
- Trực quan hóa mùa vụ bằng heatmap
- Vẽ xu hướng mùa vụ của các combo quan trọng
- Đưa ra nhận định về mùa vụ của các combo sản phẩm


## Parameters


In [ ]:
# PARAMETERS (for papermill)

# Đường dẫn dữ liệu đã làm sạch
CLEANED_DATA_PATH = "data/processed/cleaned_uk_data.csv"

# Đơn vị thời gian phân tích: 'month' hoặc 'quarter'
TIME_PERIOD = "month"  # hoặc "quarter"

# Tham số cho Apriori
MIN_SUPPORT = 0.01
MAX_LEN = 3
MIN_CONFIDENCE = 0.3
MIN_LIFT = 1.2

# Tham số cho phân tích mùa vụ
TOP_N_RULES_HEATMAP = 20  # Số combo top để hiển thị trong heatmap
MIN_PERIODS_FOR_SEASONALITY = 2  # Số giai đoạn tối thiểu để coi là có mùa vụ
VARIANCE_THRESHOLD = 0.3  # Ngưỡng hệ số biến thiên để xác định mùa vụ

# Bật/tắt các biểu đồ
PLOT_HEATMAP = True
PLOT_TRENDS = True
PLOT_SEASONAL_ANALYSIS = True


## Set up


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from apriori_library import DataVisualizer

# Thiết lập style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

visualizer = DataVisualizer()


## Tải dữ liệu


In [ ]:
# Đọc dữ liệu đã làm sạch
df_clean = pd.read_csv(CLEANED_DATA_PATH, parse_dates=["InvoiceDate"])

print("=== Thông tin dữ liệu ===")
print(f"- Số giao dịch: {df_clean.shape[0]:,}")
print(f"- Khoảng thời gian: {df_clean['InvoiceDate'].min()} đến {df_clean['InvoiceDate'].max()}")
print(f"- Số hoá đơn: {df_clean['InvoiceNo'].nunique():,}")
print(f"- Số sản phẩm: {df_clean['Description'].nunique():,}")

df_clean.head()


## Phân tích luật kết hợp theo thời gian

Để xác định mùa vụ, chúng ta cần phân tích luật kết hợp cho từng giai đoạn thời gian (tháng/quý).


In [ ]:
print(f"Đang phân tích luật kết hợp theo {TIME_PERIOD}...")
print("Quá trình này có thể mất vài phút...")

temporal_rules = visualizer.analyze_temporal_rules(
    df=df_clean,
    invoice_col="InvoiceNo",
    item_col="Description",
    date_col="InvoiceDate",
    time_period=TIME_PERIOD,
    min_support=MIN_SUPPORT,
    max_len=MAX_LEN,
    min_confidence=MIN_CONFIDENCE,
    min_lift=MIN_LIFT,
)

print(f"\n=== Kết quả phân tích ===")
print(f"- Số giai đoạn đã phân tích: {len(temporal_rules)}")
print(f"- Các giai đoạn: {sorted(temporal_rules.keys())}")

for period in sorted(temporal_rules.keys()):
    print(f"  - {period}: {len(temporal_rules[period])} luật")


## Heatmap mùa vụ của các combo sản phẩm

Heatmap này cho thấy độ mạnh (lift) của các combo sản phẩm qua các giai đoạn thời gian. Màu sắc càng đậm cho thấy combo càng mạnh trong giai đoạn đó.


In [ ]:
if PLOT_HEATMAP and temporal_rules:
    print("Đang vẽ heatmap mùa vụ...")
    visualizer.plot_seasonality_heatmap(
        temporal_rules=temporal_rules,
        top_n_rules=TOP_N_RULES_HEATMAP,
        metric="lift",
        title=f"Mùa vụ của các combo sản phẩm (Top {TOP_N_RULES_HEATMAP} theo Lift)",
    )


## Xác định các combo sản phẩm có tính mùa vụ

Combo có tính mùa vụ là những combo có hệ số biến thiên (CV) cao, nghĩa là độ mạnh của chúng thay đổi đáng kể giữa các giai đoạn.


In [ ]:
seasonal_combos = visualizer.identify_seasonal_combos(
    temporal_rules=temporal_rules,
    min_periods=MIN_PERIODS_FOR_SEASONALITY,
    variance_threshold=VARIANCE_THRESHOLD,
)

print("=== Các combo sản phẩm có tính mùa vụ ===")
print(f"Số combo có tính mùa vụ: {len(seasonal_combos)}")

if not seasonal_combos.empty:
    display(seasonal_combos.head(20))
    
    if PLOT_SEASONAL_ANALYSIS:
        # Vẽ biểu đồ top combo có mùa vụ
        top_seasonal = seasonal_combos.head(10)
        
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # Hệ số biến thiên Lift
        axes[0].barh(
            top_seasonal["Combo"].str[:50],  # Rút ngắn tên để hiển thị
            top_seasonal["Lift CV"],
            color="coral"
        )
        axes[0].set_title("Top 10 combo có hệ số biến thiên Lift cao nhất")
        axes[0].set_xlabel("Hệ số biến thiên (CV) của Lift")
        axes[0].set_ylabel("Combo sản phẩm")
        
        # Lift min vs max
        x_pos = np.arange(len(top_seasonal))
        width = 0.35
        axes[1].bar(x_pos - width/2, top_seasonal["Lift min"], width, 
                    label="Lift min", color="lightblue")
        axes[1].bar(x_pos + width/2, top_seasonal["Lift max"], width, 
                    label="Lift max", color="steelblue")
        axes[1].set_title("Phạm vi Lift của các combo có mùa vụ")
        axes[1].set_xlabel("Combo sản phẩm")
        axes[1].set_ylabel("Lift")
        axes[1].set_xticks(x_pos)
        axes[1].set_xticklabels(
            [combo[:30] + "..." for combo in top_seasonal["Combo"]], 
            rotation=45, ha="right"
        )
        axes[1].legend()
        axes[1].grid(True, alpha=0.3, axis="y")
        
        plt.tight_layout()
        plt.show()
else:
    print("Không tìm thấy combo nào có tính mùa vụ rõ ràng.")


## Xu hướng mùa vụ của các combo quan trọng

Biểu đồ xu hướng cho thấy sự thay đổi độ mạnh (lift) của các combo qua các giai đoạn thời gian.


In [ ]:
if PLOT_TRENDS and not seasonal_combos.empty:
    # Chọn top 5 combo có mùa vụ để vẽ xu hướng
    top_5_seasonal = seasonal_combos.head(5)
    
    print("=== Xu hướng mùa vụ của top 5 combo ===")
    
    for idx, row in top_5_seasonal.iterrows():
        rule_str = row["Combo"]
        print(f"\nĐang vẽ xu hướng: {rule_str}")
        visualizer.plot_seasonality_trend(
            temporal_rules=temporal_rules,
            rule_str=rule_str,
            metric="lift",
        )


## Nhận định về mùa vụ của các combo sản phẩm

Dựa trên phân tích trên, chúng ta có thể đưa ra các nhận định sau:

### 1. Các combo có tính mùa vụ cao

Những combo có hệ số biến thiên (CV) lớn cho thấy:
- Chúng mạnh yếu khác nhau đáng kể giữa các giai đoạn
- Có thể liên quan đến các sự kiện, mùa lễ, hoặc xu hướng thời tiết
- Cần được quản lý đặc biệt trong chiến lược kinh doanh

### 2. Giai đoạn mạnh nhất

Từ heatmap và xu hướng, chúng ta có thể xác định:
- Giai đoạn nào có nhiều combo mạnh nhất
- Giai đoạn nào combo có lift cao nhất
- Pattern theo mùa (ví dụ: mùa đông, mùa hè, mùa lễ)

### 3. Xu hướng theo thời gian

Một số combo có thể:
- Tăng mạnh vào các tháng/quý cụ thể (ví dụ: mùa lễ, mùa hè, v.v.)
- Giảm mạnh vào các giai đoạn khác
- Có pattern lặp lại hàng năm

### 4. Ứng dụng thực tế

**Tối ưu hóa chiến lược kinh doanh:**
- Cross-selling theo mùa: Tập trung vào các combo mùa vụ trong giai đoạn chúng mạnh nhất
- Dự trữ hàng tồn kho: Tăng dự trữ các sản phẩm trong combo mùa vụ trước giai đoạn cao điểm
- Thiết kế chương trình khuyến mãi: Tạo combo deal theo mùa dựa trên các combo có mùa vụ
- Quản lý kệ hàng: Sắp xếp các sản phẩm trong combo mùa vụ gần nhau trong giai đoạn cao điểm

### 5. Kết luận

Phân tích mùa vụ giúp doanh nghiệp:
- Hiểu rõ hơn về hành vi mua hàng theo thời gian
- Tối ưu hóa chiến lược bán hàng và marketing
- Cải thiện quản lý hàng tồn kho
- Tăng doanh thu thông qua cross-selling hiệu quả
